In [1]:
import numpy as np
import pandas as pd
import pooch
from scipy.optimize import root

In [2]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [3]:
bc_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1850'].values[0]
oc_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1850'].values[0]
so2_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1850'].values[0]

bc_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'2014'].values[0]
oc_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'2014'].values[0]
so2_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'2014'].values[0]

In [4]:
bc_1850

2.5711244787427185

In [5]:
bc_2014

9.744379658087515

In [6]:
# I'm not sure where these numbers came from now!
target_erfari_so2 = -0.49
target_erfari_bc = +0.38
target_erfari_oc = -0.14
target_erfaci_so2 = -0.88
target_erfaci_bc = -0.01
target_erfaci_oc = -0.08
target_erfaci = -0.84

In [7]:
def erf_rootfinder(x):
    alpha_so2, alpha_bc, alpha_oc, s_so2, s_bc, s_oc, beta = x
    erfari_so2 = (so2_2014 - so2_1850) * alpha_so2
    erfari_bc = (bc_2014 - bc_1850) * alpha_bc
    erfari_oc = (oc_2014 - oc_1850) * alpha_oc
    erfaci_so2 = beta * (np.log(1 + s_so2 * so2_2014) - np.log(1 + s_so2 * so2_1850))
    erfaci_bc = beta * (np.log(1 + s_bc * bc_2014) - np.log(1 + s_bc * bc_1850))
    erfaci_oc = beta * (np.log(1 + s_oc * oc_2014) - np.log(1 + s_oc * oc_1850))
    erfaci = beta * (np.log(1 + s_so2 * so2_2014 + s_bc * bc_2014 + s_oc * oc_2014) - np.log(1 + s_so2 * so2_1850 + s_bc * bc_1850 + s_oc * oc_1850))
    return np.array(
        [
            erfari_so2 - target_erfari_so2, 
            erfari_bc - target_erfari_bc, 
            erfari_oc - target_erfari_oc, 
            erfaci_so2 - target_erfaci_so2, 
            erfaci_bc - target_erfaci_bc, 
            erfaci_oc - target_erfaci_oc, 
            erfaci - target_erfaci
        ]
    )

In [8]:
root(
    erf_rootfinder, 
    np.array(
        [
            -0.0043,
            0.05158,
            -0.0052,
            0.037031928,
            0.001,
            0.016231426,
            -0.735769463,
        ],
    ),
    method='lm',
    options={'maxiter': 500000}
)

   cov_x: array([[ 8.34556608e-05, -0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  1.94342346e-02, -0.00000000e+00,
         1.47199693e-15,  4.96684722e-17,  1.92244021e-16,
         1.39268617e-14],
       [ 0.00000000e+00,  0.00000000e+00,  3.11438732e-03,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  1.47199693e-15,  0.00000000e+00,
         3.25254174e+00,  1.09610011e-01,  4.26720197e-01,
         3.07416899e+01],
       [ 0.00000000e+00,  4.96684722e-17,  0.00000000e+00,
         1.09610011e-01,  5.43113288e-02,  1.43466473e-02,
         1.03729345e+00],
       [ 0.00000000e+00,  1.92244021e-16,  0.00000000e+00,
         4.26720197e-01,  1.43466473e-02,  7.06608527e-02,
         4.01489023e+00],
       [ 0.00000000e+00,  1.39268617e-14,  0.00000000e+00,
         3.07416899e+01,  1.03729345e+00,  4.01489023e+00,
        